# Imports

In [1]:
from utils import *
from utils2 import *
from utils3 import *
from gpcrdb_soup import *

In [2]:
import Bio
import yaml
import os
from functools import partial
from operator import is_not
from tqdm import tqdm

In [3]:
def disp3(df: pd.DataFrame):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df.head(3))

# GPCRDB

In [4]:
table = get_table(reload=True)

In [5]:
disp3(table)

,filler,uniprot(gene),filler2,receptor family,Cl.,Species,Method,PDB,Refined Structure,Resolution,Preferred Chain,State,Degree active %,% of Seq1,Family,Subtype,Note,% of Seq2,Fusion,Antibodies,Name1,Type1,Function,Name2,Type2,D2x50 - S3x39,Sodium in structure,Authors,Reference,PDB date,Annotated,pdb_link
0,,STE2,STE2,STE2,D1(Ste2-likefungalpheromone),strainATCC204508/S288c,cryo-EM,7AD3,-,3.5,A,Active,-,69.0,GPa1family,αPa1,TwoalphasubunitsboundtoSTE2dimer,29,-,-,Alpha-factormatingpheromonepubchem,peptide,Agonist,-,-,No,No,"VelazhahanV,MaN,Pándy-SzekeresG,KooistraAJ,Lee...",10.1038/S41586-020-2994-1,2020-12-09,Yes,https://files.rcsb.org/download/7AD3.pdb
1,,CCR2,CCR2,Chemokine,A(Rhodopsin),Human,X-ray,5T1A,-,2.8,A,Inactive,23,71.0,-,-,-,-,T4-Lysozyme,-,BMS-681pubchemGTPL9431pubchem,smallmoleculesmallmolecule,AntagonistNAM,"CCL16,CCL7,CCL13,CCL8,CCL24,CCL2,CCL11,CCL26",protein,No,No,"ZhengY,QinL,ZacaríasNVO,deVriesH,HanGW,Gustavs...",10.1038/NATURE20605,2016-12-14,Yes,https://files.rcsb.org/download/5T1A.pdb
2,,OPRM,μ,Opioid,A(Rhodopsin),Mouse,X-ray,4DKL,-,2.8,A,Inactive,7,71.0,-,-,-,-,T4-Lysozyme,-,"Methyl4-[[(4R,4aS,7R,7aR,12bS)-3-(cyclopropylm...",smallmolecule,Antagonist,"[Leu]enkephalin,[Met]enkephalin,dynorphinA-(1-...",peptide,Yes,No,"ManglikA,KruseAC,KobilkaTS,ThianFS,MathiesenJM...",10.1038/NATURE10954,2012-03-21,Yes,https://files.rcsb.org/download/4DKL.pdb


In [6]:
# t
# Todo: https://www.ebi.ac.uk/training/online/sites/ebi.ac.uk.training.online/files/UniProt_programmatically_py3.pdf

In [7]:
df = table

# This is the thing we might want to specify with argparse
filtered = df[df['Cl.'].str.contains('A')]
filtered = filtered[filtered['Species'].str.contains('Human')]

In [8]:
# filtered

In [9]:
# 2.1) find active state (complex i.p.)
filtered_complex = filtered[filtered['Family'].str.contains('Gs')]
active = filtered_complex[filtered_complex['State'].str.contains('Active')]

In [10]:
# disp3(active)

In [11]:
# todo: check if file already exists and use overwrite=True/False
"""
for url in tqdm(active['pdb_link']):
    download_pdb(url, 'active')
"""

"\nfor url in tqdm(active['pdb_link']):\n    download_pdb(url, 'active')\n"

In [12]:
# find inactive counter parts of the active proteins ==> use gene/uniprot, Family and species to match them

genes = list(set(active['uniprot(gene)'].values.tolist()))
print(genes)

inactive = filtered[(filtered['uniprot(gene)'].isin(genes))
                    & (filtered['State'].str.contains('Inactive')) 
                    & (filtered['Species'].str.contains('Human'))]

['PE2R4', 'AA2AR', 'ADRB2', 'DRD1', 'GPR52', 'GPBAR']


In [13]:
"""for url in tqdm(inactive['pdb_link']):
    download_pdb(url, 'inactive')"""

"for url in tqdm(inactive['pdb_link']):\n    download_pdb(url, 'inactive')"

In [14]:
# disp3(inactive)

## GPCRDB

In [15]:
# Solution: manual download for the entire A class
al = load_alignment_()

In [16]:
al_df, positions = clean_alignment(al)

cleaning alignment


In [17]:
disp3(al_df)

,ID,clean_id,TM7_combined,TM7_clean,H8_combined,H8_clean,roi_pos,roi_seq
1,[Human] 5-HT1A receptor,5-HT1A receptor,_______PTLLGAIINWLGYSNSLLNPVIYAYF___,PTLLGAIINWLGYSNSLLNPVIYAYF,___NKDFQNAFKKII______________,NKDFQNAFKKII,"[7.31x30, 7.32x31, 7.33x32, 7.34x33, 7.35x34, ...",PTLLGAIINWLGYSNSLLNPVIYAYFNKDFQNAFKKII
2,[Human] 4iaq,4iaq,_______HLAIFDFFTWLGYLNSLINPIIYTMS___,HLAIFDFFTWLGYLNSLINPIIYTMS,___NEDFKQAFHKLI______________,NEDFKQAFHKLI,"[7.31x30, 7.32x31, 7.33x32, 7.34x33, 7.35x34, ...",HLAIFDFFTWLGYLNSLINPIIYTMSNEDFKQAFHKLI
3,[Human] 4iar,4iar,_______HLAIFDFFTWLGYLNSLINPIIYTM____,HLAIFDFFTWLGYLNSLINPIIYTM,___NEDFKQAFHKLI______________,NEDFKQAFHKLI,"[7.31x30, 7.32x31, 7.33x32, 7.34x33, 7.35x34, ...",HLAIFDFFTWLGYLNSLINPIIYTMNEDFKQAFHKLI


# Data loading

## PDB

In [18]:
files_a, prots_a = get_pdb_files(path='data/pdb/active/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_a), prots_a))

Found 13 proteins: ['7BW0', '7CFM', '7D7M', '7BZ2', '3SN6', '7DHR', '6E67', '5G53', '6GDG', '7CFN', '7DHI', '6LI3', '6NI3'].


In [19]:
# Data loading
files_i, prots_i = get_pdb_files(path='data/pdb/inactive/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_i), prots_i))

Found 74 proteins: ['5D5B', '3D4S', '2R4S', '3NY8', '3NYA', '3PDS', '5K2C', '5MZJ', '6PS4', '5JTB', '5K2B', '3RFM', '6PS0', '6JZH', '5D6L', '5IU7', '6OBA', '6MH8', '6AQF', '6PS2', '3REY', '6PS1', '6PS6', '3KJ6', '3NY9', '5OM4', '5YHL', '6GT3', '6PS3', '6LPJ', '5IU8', '3UZC', '5IU4', '5OLH', '5VRA', '6ZDV', '5IUB', '5OLZ', '5K2A', '5NM4', '3EML', '3VG9', '5OLO', '5YWY', '5UVI', '5OLV', '4GBR', '3VGA', '5NLX', '5OLG', '3UZA', '6LPL', '2R4R', '5JQH', '5NM2', '4EIY', '6PRZ', '6PS7', '6WQA', '3PWH', '6PS5', '6LPK', '5UIG', '5X7D', '6ZDR', '5N2R', '6S0L', '5MZP', '5IUA', '5OM1', '5D5A', '6S0Q', '2RH1', '5K2D'].


In [20]:
try:
    seg_aligns = get_alignment(prots_a)
    print(yaml.dump(seg_aligns, default_flow_style=False))
except:
    print("Doesnt work anymore...")

Searching for alignment: https://gpcrdb.org/services/alignment/protein/7BW0,7CFM,7D7M,7BZ2,3SN6,7DHR,6E67,5G53,6GDG,7CFN,7DHI,6LI3,6NI3/TM7/
Doesnt work anymore...


In [21]:
# since the identifiers are only human readable...

In [22]:
genes = list(al_df['clean_id'])

l4=[]
l5=[]
l6=[]

for g in genes:
    if len(g) == 4:
        l4.append(g)
    elif len(g) == 5:
        l5.append(g)
    elif len(g) == 6:
        l6.append(g)
    else:
        pass

In [23]:
print(l4)  # these are (all?) pdbs

['4iaq', '4iar', '5v54', '6g79', '7c61', '6a93', '6a94', '6wgt', '6wh4', '6wha', '4ib4', '4nc3', '5tud', '5tvn', '6drx', '6dry', '6drz', '6ds0', '6bqg', '6bqh', '5cxv', '6oij', '6wjc', '3uon', '4mqs', '4mqt', '5yc8', '5zk3', '5zk8', '5zkb', '5zkc', '6oik', '6u1n', '5dsg', '6kp6', '6ol9', '6kux', '6kuy', '6k41', '6k42', '6kuw', '2r4r', '2r4s', '2rh1', '3d4s', '3kj6', '3ny8', '3ny9', '3nya', '3p0g', '3pds', '3sn6', '4gbr', '4lde', '4ldl', '4ldo', '4qkx', '5d5a', '5d5b', '5d6l', '5jqh', '5x7d', '6e67', '6mxt', '6n48', '6ni3', '6oba', '6prz', '6ps0', '6ps1', '6ps2', '6ps3', '6ps4', '6ps5', '6ps6', '7bz2', '7dhi', '7dhr', '6cm4', '6luq', '6vms', '7dfp', '3pbl', '5wiu', '5wiv', '3rze', '4yay', '4zud', '6do1', '6os0', '6os1', '6os2', '5unf', '5ung', '5unh', '5xjm', '6jod', '7c6a', '5vbl', '6knm', '5o9h', '6c1q', '6c1r', '5glh', '5gli', '5x93', '5xpr', '6igk', '6igl', '6k1q', '6lry', 'FPR1', '6lw5', '6omm', 'FPR3', '6ko5', '7br3', '6w25', '5zbh', '5zbq', '7ddz', '6os9', '6osa', '6pwc', '6up7',

In [24]:
# print(l5)  # these are?

In [25]:
# print(l6)  # another id..

In [26]:
# how did i load active inactive again?

In [27]:
# This should be put into a function somewhere or just left out!
inactive['pdb_lower'] = inactive.PDB.apply(lambda x: x.lower())
active['pdb_lower'] = active.PDB.apply(lambda x: x.lower())

<ipython-input-27-29d6b45c2935>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive['pdb_lower'] = inactive.PDB.apply(lambda x: x.lower())


In [28]:
found = []
for g in l4:
    select = inactive[inactive['pdb_lower'].str.contains(g.lower())]
    if len(select) > 0:
        found.append(g)

In [29]:
print("Found {} / {} inactive proteins.".format(len(found), len(inactive)))

Found 74 / 74 inactive proteins.


In [30]:
# check
# print(sorted(list(inactive['PDB'].str.lower()))[:10])
# print(sorted(l4)[:10])

# Structures (PDB)

## Combine gpcrdb-table, pdb-structures and alignments

In [31]:
# FIX: DO NOT USE ONLY CHAIN ID 0 ---> USE ALL CHAINS AND USE SEQ ALIGNMENT TO GET THE CORRECT CHAIN

In [32]:
structures_a = pdb_data(files_a)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  4.56it/s]


In [33]:
disp3(structures_a)

,prot_id,prot_len,prot_seq,pp_ids,pp_lens,pp_seqs,psi_phi
0,3SN6,349,"[T, E, D, Q, R, N, E, E, K, A, Q, R, E, A, N, ...","[0, 1, 2, 3]","[51, 115, 51, 132]",[TEDQRNEEKAQREANKKIEKQLQKDKQVYRATHRLLLLGAGESGK...,"[[[None, -2.4757948598365656], [0.838827129769..."
1,5G53,286,"[S, S, V, Y, I, T, V, E, L, A, I, A, V, L, A, ...","[0, 1, 2]","[141, 53, 89]",[SSVYITVELAIAVLAILGNVLVCWAVWLNSNLQNVTNYFVVSLAA...,"[[[None, 2.349491635258823], [-1.0346566470181..."
2,6E67,458,"[D, E, V, W, V, V, G, M, G, I, V, M, S, L, I, ...",[0],[457],[DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFIT...,"[[[None, 2.4471797633510577], [-1.186672557221..."


In [34]:
full_a = structure_to_full(active, structures_a, al_df)

In [35]:
full_a.to_pickle('full_a.pkl')

In [36]:
structures_i = pdb_data(files_i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:13<00:00,  5.37it/s]


In [37]:
full_i = structure_to_full(inactive, structures_i, al_df)

In [38]:
full_i.to_pickle('full_i.pkl')

## Load Full

In [39]:
full_a = pd.read_pickle('full_a.pkl')

In [40]:
full_a = full_a[full_a['uniprotid']!='RESIDUES'].dropna(subset=['uniprotid'])

In [41]:
len(full_a)

11

In [42]:
full_i = pd.read_pickle('full_i.pkl')

In [43]:
full_i = full_i[full_i['uniprotid']!='RESIDUES']

In [44]:
len(full_i)

66

In [107]:
# Include Uniprot data
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import requests as r
from Bio import SeqIO
from io import StringIO



def get_uniprot(query='',query_type='PDB_ID'):
    #query_type must be: "PDB_ID" or "ACC"
    url = 'https://www.uniprot.org/' #This is the webser to retrieve the Uniprot data
    params = {
    'from':query_type,
    'to':'ACC',
    'format':'txt',
    'query':query
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('ascii')
    request = urllib.request.Request(url, data)
    with urllib.request.urlopen(request) as response:
        res = response.read()
        page=BeautifulSoup(res).get_text()
        page=page.splitlines()
    return page

def get_sequence_name(cID):
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)

    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,'fasta'))
    return str(pSeq[0].seq), pSeq[0].name

In [108]:
eg = get_uniprot('6li3')

In [109]:
pdbtouniprot('6li3')

'Q9Y2T5'

In [110]:
get_sequence_name('Q9Y2T5')

('MNESRWTEWRILNMSSGIVNVSERHSCPLGFGHYSVVDVCIFETVVIVLLTFLIIAGNLTVIFVFHCAPLLHHYTTSYFIQTMAYADLFVGVSCLVPTLSLLHYSTGVHESLTCQVFGYIISVLKSVSMACLACISVDRYLAITKPLSYNQLVTPCRLRICIILIWIYSCLIFLPSFFGWGKPGYHGDIFEWCATSWLTSAYFTGFIVCLLYAPAAFVVCFTYFHIFKICRQHTKEINDRRARFPSHEVDSSRETGHSPDRRYAMVLFRITSVFYMLWLPYIIYFLLESSRVLDNPTLSFLTTWLAISNSFCNCVIYSLSNSVFRLGLRRLSETMCTSCMCVKDQEAQEPKPRKRANSCSI',
 'sp|Q9Y2T5|GPR52_HUMAN')

In [48]:
import re


searching = False
regex = '^[a-zA-Z].{2}[ ].{3}'
for _ in eg:
    if 'SQ   ' in _:
        searching = True
        print(_)
    elif searching == True:
        if re.search(regex, _) and (not 'SQ   ' in _):
            searching = False
        else:
            print(_)

In [49]:
def add_uniprot_topology(df):
    def get_topo(x):
        domains=[]
        positions=[]
        data=get_uniprot(query=x, query_type='ACC')
        add_next_note = False
        for line in data:
            if 'FT   ' in line:
                line=line.strip().replace('FT   ','')
                if ('TOPO_DOM' in line) or ('TRANSMEM        ' in line):
                    add_next_note=True
                    line = line.replace('TOPO_DOM        ', '').replace('TRANSMEM        ', '')
                    positions.append((line.split('.')[0], line.split('.')[2]))
                if ('note' in line) and add_next_note is True:
                    line = line.replace('                /note=', '').replace('\"','')
                    domains.append(line.replace('                /note=', '').replace('\"',''))
                    add_next_note=False
        return [domains, positions]
    lambdafunc = lambda x: pd.Series([get_topo(x.uniprotid)[0],
                                      get_topo(x.uniprotid)[1]])
    df[['Topology', 'topo_pos']] = df.apply(lambdafunc, axis=1).copy()
    return df

In [50]:
full_a = add_uniprot_topology(full_a)

In [51]:
full_i = add_uniprot_topology(full_i)

In [52]:
full_i['Topology'][:5]

4    []
5    []
6    []
7    []
8    []
Name: Topology, dtype: object

In [53]:
full_i['topo_pos'][:5]

4    []
5    []
6    []
7    []
8    []
Name: topo_pos, dtype: object

In [54]:
# Include Sifts data (which regions are included)

In [55]:
from utils3 import *

In [56]:
def get_maps_(full):
    l = []
    for i in range(len(full)):
        uniprot_mappings = list_uniprot_pdb_mappings(full['pdb_lower'].iloc[i])[full['uniprotid'].iloc[i]]['mappings'][0]
        l.append(uniprot_mappings)
    return l

In [57]:
def get_maps(pdbs, mappings):
    cols = ['pdb_lower', 'start_pdb', 'end_pdb', 'start_uniprot', 'end_uniprot']
    maps = {}
    for _ in range(len(mappings)):
        start = l[_]['start']['residue_number']
        end = l[_]['end']['residue_number']
        unp_start = l[_]['unp_start']
        unp_end = l[_]['unp_end']
        data = [pdbs[_], start, end, unp_start, unp_end]
        maps.update({_: data})
    return pd.DataFrame.from_dict(data=maps).T.rename(columns = {k:cols[k] for k in range(len(cols))})   

In [58]:
# l_a = get_maps_(full_a)

In [59]:
# l_i = get_maps_(full_i)

In [60]:
# mapping_i = get_maps(full_i['pdb_lower'].tolist(), l_i)

In [61]:
# full_i_ = pd.merge(full_i, mapping_i, on='pdb_lower')

In [62]:
# full_a['pdb_lower']

In [63]:
# mapping_a = get_maps(full_a['pdb_lower'].tolist(), l_a)

In [64]:
# full_a_ = pd.merge(full_a, mapping_a, on='pdb_lower')

In [65]:
# disp3(full_i_)

In [66]:
# check if these numbers mean anything

In [67]:
# 1 get uniprot protein sequence ==> done

In [68]:
# 2 compare full sequences  ==> wtf

In [69]:
# 3 compare sections that supposedly are "mapped"  ==> idk what is going on (well at least 1 of the 4 seqs i found on uniprot 
# corresponds to my true sequence (kinda))

In [70]:
# This should let us evaluate where we have missing protein sections ==> bett

In [71]:
# 4 for each residue get the position (dont need anymore since we have )

In [72]:
# 5 for each residue get the angles

In [73]:
# 6 select target region based on position

In [74]:
# 7 full position array  (tbd?)

In [75]:
# 8 pad the selected angles

In [76]:
# 9 make a sparse plot

In [77]:
# todo: select only the mappings corresponding to the uniprot-id we already have!

In [78]:
def align_seg_to_seq(z, padding=None, padding_l=None, padding_r=0):
    X = z[0]
    Y = z[1]
    pdb = z[2]
    if (len(X) == 0) or (len(Y) == 0):
        return None, None, None, None, None, None, None
    a_ = pairwise2.align.globalms(X, Y, 3, -.5, -.1, -0.1)[0]
    score = a_.score
    score /= len(Y)  # Is this a balanced representation?
    matching = [0 if (a_.seqB[i] == '-') else 1 for i in range(len(a_.seqB))]
    # get mean
    res_id = [idx for idx, val in enumerate(matching) if val != 0]
    res_arr = np.asarray(res_id)
    mean = np.mean(res_arr)
    # get std
    std = np.std(res_arr)
    # get start
    start = matching.index(1)
    # get end
    end = len(matching) - matching[::-1].index(1)
    if padding != None:
        start = max(0, start-padding)
        end = min(len(X), end+padding)
    if padding_l != None:
        start = max(0, start-padding_l)
    if padding_r != None:
        end = min(len(X), end+padding_r)
    return start, end, mean, std, score, res_id, pdb


def get_align_dict(full: pd.DataFrame, section_name='TM7'):
    l_seq = list(full['full_prot_seq'])
    l_seg = list(full[section_name+'_clean'])
    pdb = list(full['PDB'])
    cols = ['start', 'end', 'mean', 'std', 'score', 'res', 'PDB']
    cols = [section_name+'_'+x  if x != 'PDB' else 'PDB' for x in cols]
    l = []
    for z in zip(l_seq, l_seg, pdb):
        values = align_seg_to_seq(z)
        zipped = zip(cols, values)
        a_s = dict(zipped)
        l.append(a_s)
    a_df = pd.DataFrame(columns=cols)
    a_df = a_df.append(l, True)
    return full.merge(a_df, how='inner', left_on='PDB', right_on='PDB')  # ignore_index=True


def df_align_func(df, section_name, max_std=100, min_score=0.0):
    def align(x, max_std, min_score):
        seg = x[1]
        pps = x[0]
        min_std = 100
        start_best = 0
        end_best = 0
        # print("\nTrying to find \n{}\nFOUND".format(seg))
        pp_id=0
        for p, pp in enumerate(pps):
            if len(pp) == 0 or len(seg) == 0:
                continue
            else:
                if pp.find(seg) >= 0:
                    #print("PERFECT MATCH!!!!!!!!!!!!")
                    start = pp.find(seg)
                    return start, start+len(seg), (start+len(seg)), -1, -1, p
                a_ = pairwise2.align.globalms(seg, pp, 3, -.5, -.1, -0.1)[0]
                # get std
                score = a_.score
                score /= len(pp)
                matching = [0 if (a_.seqB[i] == '-') else 1 for i in range(len(a_.seqB))]
                # get mean
                res_id = [idx for idx, val in enumerate(matching) if val != 0]
                res_arr = np.asarray(res_id)
                std = np.std(res_arr)
                if std < min_std:
                    pp_id = 0
                    min_std = std
                    best_score = score
                    mean_best = np.mean(res_arr)
                    start_best = matching.index(1)
                    end_best = len(matching) - matching[::-1].index(1)
        # print(pps[pp_id][start_best:end_best])
        if (start_best >= 0) & (min_std < max_std):
            return start_best, end_best, mean_best, min_std, best_score, int(pp_id)
        else:
            return None, None, None, None, None, -1
    cols = ['start', 'end', 'mean', 'std', 'score', 'pp_id']
    cols = [section_name+'_'+x for x in cols]
    df[cols] = df.apply(lambda x: align([x.pp_seqs, x[section_name+'_clean']], max_std, min_score), axis=1, result_type="expand")
    df = df.astype({section_name+'_pp_id': int})
    return df
    
    
    
    

In [79]:
#del complete_i 
complete_i = df_align_func(full_i, section_name='TM7')
complete_i = df_align_func(complete_i, section_name='H8')

In [80]:
# del complete_a
complete_a = df_align_func(full_a, section_name='TM7')
complete_a = df_align_func(complete_a, section_name='H8')

In [81]:
def test_found_seg(df, section_name, filter_bad_aligns=False, max_diff=5):
    def get_seg_seq(x):
        seqs = x[0]
        if x[1] == -1:
            return ''
        else:
            seq_id = int(x[1])
            start = int(x[2])
            end = int(x[3])
            return seqs[seq_id][start:end]
    df = df[df[section_name+'_pp_id']!=-1]
    df[[section_name + '_found', section_name + '_angles']] = df.apply(
        lambda x: get_seg_seq([x.pp_seqs, x[section_name + '_pp_id'], x[section_name+'_start'], x[section_name+'_end']]), axis=1, result_type="expand")
    if filter_bad_aligns:
        df = df[abs(df[section_name+'_clean'].map(len) - df[section_name+'_found'].map(len)) <= max_diff]
    return df

In [82]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

complete_i = test_found_seg(complete_i, 'TM7', True)
complete_i = test_found_seg(complete_i, 'H8', True)
complete_a = test_found_seg(complete_a, 'TM7', True)
complete_a = test_found_seg(complete_a, 'H8', True)

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
complete_i[['TM7_clean', 'TM7_found', 'H8_clean', 'H8_found']]

In [ ]:
complete_a[['TM7_clean', 'TM7_found', 'H8_clean', 'H8_found']]

In [ ]:
def get_targets_by_pos(df, pos_l=7.48, pos_r=8.51, clean=True):
    def mask_seq(seq, mask):
        out=''.join([s for s, m in zip(seq, mask) if m])
        return [out]
    def clean_pos(x):
        return np.asarray([float(y[:4]) for y in x])
    def mask_l(pos_l, x):
        y = x[1][:len(x[0])]
        mask = y>=pos_l
        output=mask_seq(list(x[0]), list(mask))
        return output, list(y[mask])
    def mask_r(pos_r, x):
        y = x[1][-len(x[0]):]
        mask = y<=pos_r
        output=mask_seq(list(x[0]), list(mask))
        return output, list(y[mask])
    if clean:
        df['roi_pos'] = df.apply(lambda x: clean_pos(x.roi_pos), axis=1)
    df[['TM7_target']] = df.apply(lambda x: mask_l(pos_l, [x.TM7_clean, x.roi_pos]), axis=1)
    df[['H8_target']] = df.apply(lambda x: mask_r(pos_r, [x.H8_clean, x.roi_pos]), axis=1)
    return df

In [ ]:
complete_i = get_targets_by_pos(complete_i, clean=True)

In [ ]:
complete_a = get_targets_by_pos(complete_a, clean=True)

In [ ]:
complete_i.psi_phi.iloc[0][0].shape

In [ ]:
def get_target_angles(df, section_name):
    def get_target_angles_(x):
        pp_id = x[2]
        angles = np.asarray(x[0][pp_id])
        pp_seqs = x[1]
        section_seq = pp_seqs[pp_id]
        target_seq = x[3][0]
        target_pos = x[3][1]
        section_start = x[4]
        section_end = x[5]
        start = section_seq.find(target_seq[0])
        end = start + len(target_seq[0])
        return [target_seq, angles[start:end, :], target_pos]
    df[section_name + '_target_angles'] = df.apply(lambda x: get_target_angles_([
        x['psi_phi'],
        x.pp_seqs,
        x[section_name + '_pp_id'],
        x[section_name + '_target'],
        x[section_name+'_start'],
        x[section_name+'_end']]),
        axis=1)
    return df

In [ ]:
complete_i = get_target_angles(complete_i, 'TM7')
complete_i = get_target_angles(complete_i, 'H8')

In [ ]:
complete_i['TM7_target_angles'].iloc[0]

In [ ]:
complete_i['H8_target_angles'].iloc[0]

In [ ]:
complete_i['H8_target_angles']

In [ ]:
complete_a = get_target_angles(complete_a, 'TM7')
complete_a = get_target_angles(complete_a, 'H8')

In [ ]:
complete_a['H8_target_angles']

In [ ]:
complete_i['TM7_target_angles']

In [ ]:
complete_a['TM7_target_angles']

In [ ]:
complete_a.to_pickle('a_angles.pkl')

In [ ]:
complete_i.to_pickle('i_angles.pkl')

# Target Sequence

In [ ]:
def complete_data(full: pd.DataFrame, max_std_alignment=None, elongate=True, padding_r=5, target='NPFIY', filter_bad_checks=False):
    # get alignment: https://towardsdatascience.com/pairwise-sequence-alignment-using-biopython-d1a9d0ba861f
    complete = get_align_dict(full)
    # filter by maximum alignment standard deviation (basically if it is wrong)
    if max_std_alignment!=None:
        complete = complete[complete['std'] < max_std_alignment]
    # replace the TM7 with an elongated version
    if elongate:
        complete['TM7_found'] = complete.apply(lambda x: x.full_prot_seq[x.start:x.end+padding_r], axis=1)
        if filter_bad_checks:
            max_diff = 15  # maximum difference in sequence lengths between detected and true TM7 region
            complete = complete[complete['TM7_combined'].map(len) + max_diff - complete['TM7_found'].map(len) >= 0]
    # extract target sequence from the TM7
    if target_by_str!=None:
        complete['target_by_str'] = complete.apply(lambda x: align_seg_to_seq([x.TM7_combined, target, x.PDB]), axis=1)
    if target_by_pos!=None:
        complete['target_by_pos'] = complete.apply(lambda x: align_seg_to_seq([x.TM7_combined, target, x.PDB]), axis=1)
    return complete

In [ ]:
complete_a = complete_data(full_a, target='NPFIY')

In [ ]:
print(len(full_a))
print(len(complete_a))

In [ ]:
disp3(complete_a)

# Complete

In [ ]:
complete_i = complete_data(full_i, max_std_alignment=None, elongate=True, padding_r=5, target='NPFIY', filter_bad_checks=True)
# this is the part where we want to increase the target range (since it is inconsistent)

In [ ]:
disp3(complete_i[['TM7_combined', 'TM7_found']])

In [ ]:
complete_a = complete_data(full_a, max_std_alignment=None, elongate=True, padding_r=5, target='NPFIY', filter_bad_checks=True)
# this is the part where we want to increase the target range (since it is inconsistent)

In [ ]:
disp3(complete_a[['TM7_combined', 'TM7_found']])

In [ ]:
# disp3(complete_i['start'])

In [ ]:
# len(complete_i[complete_i['std'] < 10])

In [ ]:
len(complete_i)

In [ ]:
# Fix that number ~ we have 73 proteins in the full datatable but end up with 87 
# after the segments alignment is complete (unfiltered)
# disp3(complete_i['TM7_combined'])

In [ ]:
disp3(complete_i[['PDB', 'TM7_combined', 'target_wrt_tm7']])

# Valid

In [ ]:
target = 'NPLIY'

In [ ]:
def get_target_df(complete: pd.DataFrame, target='NPXXY', valid=True):
    def sum_start(a, b):
        return a + b[0]
    def sum_end(a, b):
        return a + b[1]
    def extend(ls):
        is_list = isinstance(ls[0], list)
        for i, l in enumerate(ls):
            if is_list & i == 0:
                out = l
            elif is_list:
                out.extend(l)
            else:
                return ls
        return out
    def clean_pps(x):
        return ast.literal_eval(x.replace('Seq', '').replace('(', '').replace(')', ''))
    target_df = complete[['PDB', 'uniprot(gene)', 'Resolution', 'PDB date', \
                          'TM7_found', 'score', 'prot_len', 'full_prot_seq', 'start']].copy()
    target_df['pp_seqs'] = complete.apply(lambda x: clean_pps(x.pp_seqs), axis=1).copy()
    target_df['pp_seq_lens'] = target_df.apply(lambda x: [len(y) for y in x.pp_seqs], axis=1).copy()
    # target_df.loc[:, 'target_seq'] = target
    target_df['start_absolute'] = complete.apply(lambda x: sum_start(x.start, x.target_wrt_tm7), axis=1).copy()
    target_df['end_absolute'] = complete.apply(lambda x: sum_end(x.start, x.target_wrt_tm7), axis=1).copy()
    target_df['start'] = complete['target_wrt_tm7'].apply(lambda x: x[0]).copy()
    target_df['end'] = complete['target_wrt_tm7'].apply(lambda x: x[1]).copy()
    target_df['full_aligned_seg'] = target_df.apply(lambda x: x.full_prot_seq[x.start_absolute:x.end_absolute], axis=1)
    target_df['TM7_aligned_seg'] = target_df.apply(lambda x: x.TM7_found[x.start:x.end], axis=1)
    target_df['psi_phi'] = complete.apply(lambda x: extend(ast.literal_eval(x.psi_phi)), axis=1)
    if valid:
        target_df['target_angles'] = target_df.apply(lambda x: x.psi_phi[x.start_absolute:x.end_absolute], axis=1)
        target_df = target_df[target_df['full_aligned_seg']==target_df['TM7_aligned_seg']]
        return target_df  # [len(target_df['target_angles']) >= len(target)]
    else:
        return target_df

In [ ]:
valid_a = get_target_df(complete_a, target, valid=True)

In [ ]:
valid_a

In [ ]:
valid_a.to_csv('valid_a.csv', index=False)

In [ ]:
valid_i = get_target_df(complete_i, target, valid=True)
# Filter for proteins where the alignments match ~ indicates that angles are valid and the region is present
# if only valids we compare if the full aligned segment withthe TM7 aligned segment
valid_i.to_csv('valid_i.csv', index=False)

In [ ]:
disp3(valid_a)

In [ ]:
disp3(valid_i)

In [ ]:
valid_a.target_angles

In [ ]:
valid_a

In [ ]:
np.asarray(valid_i['target_angles'].iloc[0])

In [ ]:
np.asarray(valid_a['target_angles'].iloc[0])

In [ ]:
valid_i.to_csv('valid_i.csv')

In [ ]:
# j, o, u and x do not code for an amino acid (sometimes used to represent ?)

In [ ]:
len(valid_i[valid_i['full_aligned_seg']==valid_i['TM7_aligned_seg']])

In [ ]:
x = valid_i[['prot_len', 'psi_phi']]

In [ ]:
x['len_psi_phi'] = valid_i.apply(lambda x: len(x.psi_phi), axis=1)

In [ ]:
x

In [ ]:
# match each of the active complexes to all the inactive ones (based on the uniprot genes)

In [ ]:
# interaction dataframe

interaction_df = pd.DataFrame(columns=['PDB_active', 'PDB_inactive', 'phi_psi_active', 'phi_psi_inactive', 'delta_phi_psi'])
# this should be done here...